In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [2]:
import seaborn as sns
sns.set()

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from lightgbm import LGBMClassifier

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
# train data 입력시 id, target 모두 넣어주어야 한다. 
# test data 입력시 id 모두 넣어주어야 한다.
# from sklearn.preprocessing import LabelEncoder 해주어야 함
def train_model(df_train, df_test, model, path, is_submit = True, is_score = True, user_id = "id", target = "country_destination"):
    le = LabelEncoder()

    # feature, target 전처리
    if is_submit == True:
        y_train = le.fit_transform(df_train[target])
    else:
        y_train = df_train[target]
    X_train = df_train.drop([target, user_id], axis = 1)

    X_test_id = df_test[user_id] # id 따로 정의
    X_test = df_test.drop([user_id], axis = 1)

    # train data set으로 fitting
    print("model fitting 시작 !!")
    model = model.fit(X_train, y_train)
    predic_proba = model.predict_proba(X_test)
    print("model fitting 완료 !!")
    
    if is_submit == True:
        # submit data, predict data 생성
        df_submit = pd.DataFrame(columns=["id", "country"])

        ids = []
        cts = []
        for i in range(len(X_test_id)):
            idx = X_test_id.iloc[i]
            ids += [idx] * 5
            cts += le.inverse_transform(np.argsort(predic_proba[i])[::-1])[:5].tolist()

        df_submit["id"] = ids
        df_submit["country"] = cts

        df_submit.to_csv(path, index = False)
        print("csv file 제작 완료 !!")

        !kaggle competitions submit -c airbnb-recruiting-new-user-bookings -f {path} -m "Message"
    
    if is_score == True:
        print("cross valiation 시작 !!")
        print(cross_val_score(model, X_train, y_train, scoring = "neg_log_loss", cv = 3, n_jobs = 4))
        print("cross valiation 끝 !!")
    
    return model

### Load_data

In [6]:
df_train = pd.read_csv("../data/train_age_action_count_lag_input.csv")
df_test = pd.read_csv("../data/test_age_action_count_lag_input.csv")

In [7]:
df_session = pd.read_csv("../data/sessions.csv")

In [8]:
lgb = LGBMClassifier(max_depth= 4, n_jobs=4)

In [9]:
df_train.shape, df_test.shape

((213451, 522), (62096, 521))

### Make action_detail_count

In [10]:
a = df_session.groupby(["user_id", "action_detail"])["secs_elapsed"].count().unstack().fillna(0)
a = pd.DataFrame(a)
a.rename(columns = lambda x : "detail_" + x)
df_train_new = df_train.merge(a, how = "left", left_on = "id", right_index = True)
df_test_new = df_test.merge(a, how = "left", left_on = "id", right_index = True)

In [11]:
df_train_new = df_train_new.fillna(0)
df_test_new = df_test_new.fillna(0)

In [12]:
df_train_new.shape, df_test_new.shape

((213451, 677), (62096, 676))

### Merge elapse secs, device_type mode

In [13]:
df_psh_session = pd.read_csv("../data/merged_sessions.csv")

In [14]:
df_psh_session.rename(columns = {"secs_elapsed":"secs_median"}, inplace = True)

In [15]:
df_psh_session_tmp = pd.get_dummies(df_psh_session[["secs_median", "secs_sum", "secs_mean"]])

In [16]:
df_psh_session_tmp["id"] = df_psh_session["id"]

In [17]:
df_psh_session_tmp.shape

(135483, 4)

In [18]:
df_train_new = df_train_new.merge(df_psh_session_tmp, how = "left", left_on = "id", right_on = "id")

In [19]:
df_test_new = df_test_new.merge(df_psh_session_tmp, how = "left", left_on = "id", right_on = "id")

In [20]:
df_train_new = df_train_new.fillna(0)
df_test_new = df_test_new.fillna(0)

In [ ]:
df_train_new.shape, df_test_new.shape

((213451, 680), (62096, 679))

In [ ]:
train_model(df_train_new, df_test_new, lgb, "submit/submit.csv", is_score = False)

model fitting 시작 !!
model fitting 완료 !!
csv file 제작 완료 !!
